## Script to Create New `intake-esm` Collections (csv.gz) from Old Collections (nc)

This notebook assumes a netcdf collection  already exists.
I was able to create nc collection files from the YAML files in `intake-esm-collection-defs` with the following:

```
$ conda activate /glade/work/abanihi/softwares/miniconda3/envs/legacy-intake-esm/
(legacy-intake-esm) $ intake-esm-builder -cdef glade-cesm2-cmip6-collection.yaml
```

I believe that the above conda environment just needs `intake-esm` v2019.8.23;
work is in progress to be able to point `intake-esm-builder` to a directory and have it generate the `csv.gz` collection.

In [1]:
import xarray as xr

def netcdf_to_csvgz(file_in):
    """
       file_in: netcdf file generated from intake-esm-builder
    """
    df = xr.open_dataset(file_in).to_dataframe()
    df = df.drop(columns=['resource', 'resource_type', 'direct_access', 'file_basename', 'year_offset', 'sequence_order', 'grid'])
    df = df.rename(columns = {'file_fullpath' : 'path'})
    return(df)

### Generate the collection dataframe

There are two different `CESM2-CMIP6.nc` files.
The one in `~mclong/` points to data in `/glade/collections/cdg/timeseries-cmip6`; those runs are being moved to `/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6`.

There is also a collection for `CESM1-CMIP5.nc`, which assumes data has been copied from HPSS to `/glade/p/cgd/oce/projects/cesm2-marbl/intake-esm-data`.
The `get_ocn_cmip5_files.sh` script can be used to add more data to those directories.

In [2]:
CESM2_CMIP6_mclong = netcdf_to_csvgz('/glade/u/home/mclong/.intake_esm/collections/CESM2-CMIP6.nc')
print("Dataframe from Matt's netcdf file (generated from /glade/collections)\n----\n{}".format(CESM2_CMIP6_mclong))

CESM2_CMIP6 = netcdf_to_csvgz('/glade/u/home/mlevy/.intake_esm/collections/CESM2-CMIP6.nc')
print("Dataframe from Mike's netcdf file (generated from /glade/campaign on Casper)\n----\n{}".format(CESM2_CMIP6))

CESM1_CMIP5 = netcdf_to_csvgz('/glade/u/home/mlevy/.intake_esm/collections/CESM1-CMIP5.nc')
print("Dataframe from Mike's netcdf file (generated from /glade/p/cgd/oce/projects/cesm2-marbl/intake-esm-data)\n----\n{}".format(CESM1_CMIP5))


Dataframe from Matt's netcdf file (generated from /glade/collections)
----
           experiment                                           case  \
index                                                                  
0       esm-piControl    b.e21.B1850.f09_g17.CMIP6-esm-piControl.001   
1       esm-piControl    b.e21.B1850.f09_g17.CMIP6-esm-piControl.001   
2       esm-piControl    b.e21.B1850.f09_g17.CMIP6-esm-piControl.001   
3       esm-piControl    b.e21.B1850.f09_g17.CMIP6-esm-piControl.001   
4       esm-piControl    b.e21.B1850.f09_g17.CMIP6-esm-piControl.001   
...               ...                                            ...   
224694       SSP5-8.5  b.e21.BSSP585cmip6.f09_g17.CMIP6-SSP5-8.5.002   
224695       SSP5-8.5  b.e21.BSSP585cmip6.f09_g17.CMIP6-SSP5-8.5.002   
224696       SSP5-8.5  b.e21.BSSP585cmip6.f09_g17.CMIP6-SSP5-8.5.002   
224697       SSP5-8.5  b.e21.BSSP585cmip6.f09_g17.CMIP6-SSP5-8.5.002   
224698       SSP5-8.5  b.e21.BSSP585cmip6.f09_g17.CMIP6-SSP5-

### Write`JSON` and `csv` Files Defining Collection

`intake-esm` wants the dataframe written as a `csv` file (it is okay to compress with `gzip`).
Additionally, there is a JSON file that points to the `.csv.gz` file and also defines the different columns.

In [3]:
# df.to_csv('/glade/work/mlevy/intake-esm-collection/CESM1-CMIP5_only-NOT_CMORIZED.csv.gz', compression='gzip', index=False)
import os
import json

def write_collection(df, root_dir, collection_name, desc):
    """
       Write df as a csv file (file name is collection_name+'.csv.gz'; written to root_dir+'/csv.gz/')
       Write json file describing collection (file name is collection_name+'.json'; written to root_dir+'/json/')
    """

    # Make sure output directories exist
    dir_not_found = False
    csv_dir = os.path.join(root_dir, 'csv.gz')
    json_dir = os.path.join(root_dir, 'json')
    if not os.path.isdir(csv_dir):
        dir_not_found = True
        print('Can not find directory {}'.format(csv_dir))
    if not os.path.isdir(json_dir):
        dir_not_found = True
        print('Can not find directory {}'.format(json_dir))
    if dir_not_found:
        raise ValueError('Can not find needed subdirectories in {}'.format(root_dir))

    # Write csv file
    csv_file = os.path.join(csv_dir, collection_name+'.csv.gz')
    df.to_csv(csv_file, compression='gzip', index=False)

    # Write json_file
    json_file = os.path.join(json_dir, collection_name+'.json')
    collection = dict()
    collection["esmcat_version"] = "0.1.0"
    collection["id"] = collection_name
    collection["description"] = desc
    collection["catalog_file"] = csv_file
    collection["attributes"] = []
    for col in df.columns:
        collection["attributes"].append({"column_name" : col, "vocabulary" : ""})
    collection["assets"] = {"column_name" : "path", "format" : "netcdf"}
    collection["aggregation_control"] = dict()
    collection["aggregation_control"]["variable_column_name"] = "variable"
    collection["aggregation_control"]["groupby_attrs"] = ["component", "experiment", "stream"]
    collection["aggregation_control"]["aggregations"] = []
    collection["aggregation_control"]["aggregations"].append({"type" : "union", "attribute_name" : "variable"})
    collection["aggregation_control"]["aggregations"].append({"type" : "join_existing", "attribute_name" : "date_range", "options" : {"dim" : "time", "coords" : "minimal", "compat": "override"}})
    collection["aggregation_control"]["aggregations"].append({"type" : "join_new", "attribute_name" : "member_id", "options" : {"coords" : "minimal", "compat": "override"}})
    
    with open(json_file, "w") as f:
        f.write(json.dumps(collection, indent=2))

In [4]:
write_collection(CESM1_CMIP5, '/glade/work/mlevy/intake-esm-collection', 'glade-cmip5_NOT_CMORIZED',
                 desc="ESM collection for the CESM1 raw output that went into CMIP5 data. Located on glade at /glade/p/cgd/oce/projects/cesm2-marbl/intake-esm-data")
write_collection(CESM2_CMIP6, '/glade/work/mlevy/intake-esm-collection', 'campaign_via_glade-cmip6_NOT_CMORIZED',
                 desc="ESM collection for the CESM2 raw output that went into CMIP6 data. Located in campaign storage, accessible via GLADE on casper at /glade/campaign/collections/cmip/CMIP6/timeseries-cmip6")